In [1]:
using Statistics

include("ConvolutionModule.jl")  # Load the module
include("PoolingModule.jl")  # Load the module
include("FlattenModule.jl")
include("DenseModule.jl")

include("MNISTDataLoader.jl")
include("LossAndAccuracy.jl")
include("NetworkHandlers.jl")

using .ConvolutionModule, .PoolingModule, .MNISTDataLoader, .FlattenModule, .DenseModule # Use the namespace

# Load and preprocess the data
train_features, train_labels = MNISTDataLoader.load_data(:train)
train_x, train_y = MNISTDataLoader.preprocess_data(train_features, train_labels; one_hot=true)

# Create batches
batch_size = 100  # Define your desired batch size
train_data = MNISTDataLoader.batch_data((train_x, train_y), batch_size; shuffle=true)
# input_image = Float64.(input_image)

# Initialize layers
conv_layer1 = ConvolutionModule.init_conv_layer(3, 3, 1, 6, 1, 0)
pool_layer1 = PoolingModule.init_pool_layer(2, 2, 2)
conv_layer2 = ConvolutionModule.init_conv_layer(3, 3, 6, 16, 1, 0)
pool_layer2 = PoolingModule.init_pool_layer(2, 2, 2)
flatten_layer = FlattenModule.FlattenLayer()
dense_layer1 = DenseModule.init_dense_layer(400, 84, DenseModule.relu, DenseModule.relu_grad)  # Adjusted to correct input size
dense_layer2 = DenseModule.init_dense_layer(84, 10, DenseModule.identity, DenseModule.identity_grad)

# Assemble the network
network = (conv_layer1, pool_layer1, conv_layer2, pool_layer2, flatten_layer, dense_layer1, dense_layer2)

using .NetworkHandlers, .LossAndAccuracy
function train_epoch(network, inputs, targets, epochs)
    for epoch in 1:epochs
        for i in 1:size(inputs, 4)  # Iterate over each example
            input = inputs[:, :, :, i]
            target = targets[:, i]
            
            # Forward pass
            output = NetworkHandlers.forward_pass_master(network, input)

            # Calculate loss and its gradient
            loss, grad_loss = LossAndAccuracy.loss_and_accuracy(output, target)
            println("Loss at iteration $i: $loss")

            # Backward pass
            NetworkHandlers.backward_pass_master(network, grad_loss)
        end
    end
end

train_epoch(network, train_x, train_y, 1)

Output dimensions after layer Main.ConvolutionModule.ConvLayer: (26, 26, 6)
Output dimensions after layer MaxPoolLayer: (13, 13, 6)
Output dimensions after layer Main.ConvolutionModule.ConvLayer: (11, 11, 16)
Output dimensions after layer MaxPoolLayer: (5, 5, 16)
Output dimensions after layer FlattenLayer: (400, 1)
Output dimensions after layer DenseLayer: (84, 1)
Output dimensions after layer DenseLayer: (10, 1)
Loss at iteration 1: 2.3026009
Type of network before backward pass: Tuple{Main.ConvolutionModule.ConvLayer, MaxPoolLayer, Main.ConvolutionModule.ConvLayer, MaxPoolLayer, FlattenLayer, DenseLayer, DenseLayer}
doing layer DenseLayer(Float32[-0.00979371 -0.00076838006 0.0021892902 -0.005673719 0.00619282 -0.00916964 0.0042065023 0.010305134 0.012667057 -0.010908055 0.006743393 0.0003324397 -0.010962147 -0.005868128 -0.0072961724 0.010630407 0.017568689 -0.013721497 -0.0059507326 0.0029956384 -0.0036387544 -0.0073196474 -0.0083547365 -0.0071967705 0.012549233 0.010965715 0.007109

UndefVarError: UndefVarError: `backward_pass` not defined